In [4]:
import warnings
warnings.filterwarnings('ignore')

## Get API Keys

In [5]:
from utils import load_env
load_env()

In [14]:
from utils import get_tavily_api_key
TAVILY_API_KEY = get_tavily_api_key()

## LLama3.1 Model Test

In [6]:
from utils import llama31

In [7]:
messages = [
  {"role": "user",
    "content": "Who wrote the book Charlotte's Web?"}
]

In [8]:
response = llama31(messages)
print(response)

The book "Charlotte's Web" was written by E.B. White.


## brave_search buit-in tool

In [9]:
from datetime import datetime

current_date = datetime.now()
formatted_date = current_date.strftime("%d %B %Y")
print(formatted_date)

23 January 2025


In [10]:
system_prompt_content = f"""
Environment: ipython
Tools: brave_search, wolfram_alpha
Cutting Knowledge Date: December 2023
Today Date: {formatted_date}
"""

In [11]:
messages = [
    {"role": "system", "content":  system_prompt_content},
    {"role": "user",   "content": "What is the current weather in Menlo Park, California?"}
  ]

In [12]:
response = llama31(messages)
print(response)

brave_search.call(query="current weather in Menlo Park, California")


brave_seach in the built-in search tool for Llama. We need Tavily to do the search. 

### Calling the search API

In [15]:
from utils import cprint
from tavily import TavilyClient
tavily_client = TavilyClient(api_key=TAVILY_API_KEY)

result = tavily_client.search("current weather in Menlo Park, California")
cprint(result)

{
    "query": "current weather in Menlo Park, California",
    "follow_up_questions": null,
    "answer": null,
    "images": [],
    "results": [
        {
            "title": "Weather in Menlo Park, California",
            "url": "https://www.weatherapi.com/",
            "content": "{'location': {'name': 'Menlo Park', 'region': 'California', 'country': 'United States of America', 'lat': 37.4539, 'lon': -122.1811, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1737646561, 'localtime': '2025-01-23 07:36'}, 'current': {'last_updated_epoch': 1737646200, 'last_updated': '2025-01-23 07:30', 'temp_c': 9.1, 'temp_f': 48.4, 'is_day': 1, 'condition': {'text': 'Sunny', 'icon': '//cdn.weatherapi.com/weather/64x64/day/113.png', 'code': 1000}, 'wind_mph': 2.7, 'wind_kph': 4.3, 'wind_degree': 68, 'wind_dir': 'ENE', 'pressure_mb': 1027.0, 'pressure_in': 30.34, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 62, 'cloud': 0, 'feelslike_c': 9.0, 'feelslike_f': 48.1, 'windchill_c': 4.5, 'windchi

In [16]:
search_result = result["results"][0]["content"]
print(search_result)

{'location': {'name': 'Menlo Park', 'region': 'California', 'country': 'United States of America', 'lat': 37.4539, 'lon': -122.1811, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1737646561, 'localtime': '2025-01-23 07:36'}, 'current': {'last_updated_epoch': 1737646200, 'last_updated': '2025-01-23 07:30', 'temp_c': 9.1, 'temp_f': 48.4, 'is_day': 1, 'condition': {'text': 'Sunny', 'icon': '//cdn.weatherapi.com/weather/64x64/day/113.png', 'code': 1000}, 'wind_mph': 2.7, 'wind_kph': 4.3, 'wind_degree': 68, 'wind_dir': 'ENE', 'pressure_mb': 1027.0, 'pressure_in': 30.34, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 62, 'cloud': 0, 'feelslike_c': 9.0, 'feelslike_f': 48.1, 'windchill_c': 4.5, 'windchill_f': 40.1, 'heatindex_c': 5.9, 'heatindex_f': 42.6, 'dewpoint_c': 3.5, 'dewpoint_f': 38.3, 'vis_km': 16.0, 'vis_miles': 9.0, 'uv': 0.0, 'gust_mph': 5.5, 'gust_kph': 8.9}}


### Promting Llama with search tool response

In [17]:
messages = [
    {
        "role": "system",
        "content": system_prompt_content,
    },
    {
        "role": "user", 
       "content": "What is the current weather in Menlo Park, California?",
    },
    {
        "role":"assistant",
        "content": response,
    },
    {
        "role": "ipython",
        "content": search_result,
    },
]

In [18]:
response = llama31(messages)
print(response)

The current weather in Menlo Park, California is sunny with a temperature of 48.4°F (9.1°C). The wind is blowing at 4.3 km/h (2.7 mph) from the east-northeast, and the humidity is 62%.


## WolframAlpha buit-in tool